In [106]:
# Porting over the matlab code into python to see if we can reproduce results with the MNE library.

In [107]:
# Import common libraries
import pandas as pd
import numpy as np
import os
from collections import defaultdict
from copy import deepcopy

# Import MNE processing
from mne.viz import plot_compare_evokeds
from mne import Epochs, events_from_annotations, set_log_level

# Import MNE-NIRS processing
import mne
from mne_nirs.channels import get_long_channels
from mne_nirs.channels import picks_pair_to_idx
from mne_nirs.datasets import fnirs_motor_group
from mne.preprocessing.nirs import beer_lambert_law, optical_density,\
    temporal_derivative_distribution_repair, scalp_coupling_index
from mne_nirs.signal_enhancement import enhance_negative_correlation

# Scikit Learn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids

# Import StatsModels
import statsmodels.formula.api as smf

# Import Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns

In [108]:
# Tip for using machine learning for exploratory type data analysis

# Use principal component analysis (PCA) or independent component analysis (ICA), 
# to identify patterns in the fNIRS data that are not immediately obvious. 
# This can be used to identify latent neural networks or to identify different sources of signal variation.

# ==========================================================

# I'm curious how we could incorportate these into the data analsyis pipeline

# Anomaly detection
# Unsupervised learning algorithms such as one-class SVM, Autoencoder, and Isolation Forest can be used to identify outliers or abnormal patterns in the data.

# Time-series analysis
# Techniques such as time-series decomposition, ARIMA, and LSTM can be used to analyze the temporal dynamics of the fNIRS data and identify trends or patterns over time.

In [109]:
# Recurring values that we will keep the same

# Length of the measured interval
interval_length = 15
# How you would like to rename the numeric triggers from Aurora
trigger_id = {'4': 'Control', '2': 'Neutral', '3': 'Inflammatory', '1':'Practice'}
# What files would you like to ignore while looping through subjects
ignore = [".DS_Store", "sub-03"]


# Analyze Each Subjects Data Individually and Return it

In [110]:
def individual_analysis(bids_path):
    
    # Read data with annotations in BIDS format
    # raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    raw_intensity = mne.io.read_raw_snirf(bids_path, verbose=True, preload=False)
    raw_intensity = get_long_channels(raw_intensity, min_dist=0.01)
    
    channel_types = raw_intensity.copy()
    # print(channel_types)
    
    raw_intensity.annotations.rename(trigger_id)

    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))

    # Down sample and apply signal cleaning techniques
    raw_od.resample(0.8)
    raw_od = temporal_derivative_distribution_repair(raw_od)

    # Convert to haemoglobin and filter
    raw_haemo = beer_lambert_law(raw_od, ppf=0.1)
    raw_haemo = raw_haemo.filter(0.02, 0.3,
                                 h_trans_bandwidth=0.1, l_trans_bandwidth=0.01,
                                 verbose=False)

    # Apply further data cleaning techniques and extract epochs
    raw_haemo = enhance_negative_correlation(raw_haemo)
    # Extract events but ignore those with
    # the word Ends (i.e. drop ExperimentEnds events)
    events, event_dict = events_from_annotations(raw_haemo, verbose=False)
    
    # Remove all STOP triggers to hardcode duration to 30 secs per MNE specs
    events = events[::2]
    # print(events)

    epochs = Epochs(raw_haemo, events, event_id=event_dict, tmin=-1, tmax=15,
                    reject=dict(hbo=200e-6), reject_by_annotation=True,
                    proj=True, baseline=(None, 0), detrend=0,
                    preload=True, verbose=False)

    return raw_haemo, epochs

# Loop through subjects for individual analysis

In [134]:
all_evokeds = defaultdict(list)

root_dir = '../../LabResearch/IndependentStudy/DataAnalysis'

subjects = os.listdir(f'{root_dir}/BIDS_Anon/')

# print(subjects)

for sub in subjects:
    if sub not in ignore:
        print("sub", sub)
        # Create path to file based on experiment info
        f_path = f'{root_dir}/BIDS_Anon/{sub}/nirs/{sub}_task-AnonCom_nirs.snirf'

        # Analyse data and return both ROI and channel results
        raw_haemo, epochs = individual_analysis(f_path)

        for cidx, condition in enumerate(epochs.event_id):
            # all_evokeds[condition].append(epochs[condition].average())
            all_evokeds[condition].append(epochs[condition])

print(all_evokeds)


sub sub-06
Loading /Users/nolanbrady/Desktop/fNIRs-data-pipeline/python/../../LabResearch/IndependentStudy/DataAnalysis/BIDS_Anon/sub-06/nirs/sub-06_task-AnonCom_nirs.snirf
Reading 0 ... 18357  =      0.000 ...  1804.493 secs...
sub sub-07
Loading /Users/nolanbrady/Desktop/fNIRs-data-pipeline/python/../../LabResearch/IndependentStudy/DataAnalysis/BIDS_Anon/sub-07/nirs/sub-07_task-AnonCom_nirs.snirf
Reading 0 ... 17824  =      0.000 ...  1752.099 secs...
sub sub-05
Loading /Users/nolanbrady/Desktop/fNIRs-data-pipeline/python/../../LabResearch/IndependentStudy/DataAnalysis/BIDS_Anon/sub-05/nirs/sub-05_task-AnonCom_nirs.snirf
Reading 0 ... 17061  =      0.000 ...  1677.096 secs...
defaultdict(<class 'list'>, {'Control': [<Epochs |  3 events (all good), -1.25 - 15 sec, baseline -1.25 – 0 sec, ~171 kB, data loaded,
 'Control': 3>, <Epochs |  3 events (all good), -1.25 - 15 sec, baseline -1.25 – 0 sec, ~171 kB, data loaded,
 'Control': 3>, <Epochs |  3 events (all good), -1.25 - 15 sec, base

# Extract Evoked Amplitude


In [137]:

df = pd.DataFrame(columns=['ID', 'Chroma', 'Condition', 'Value'])
temporal_measurements = []

for idx, evoked in enumerate(all_evokeds):
    subj_id = 0
    for subj_data in all_evokeds[evoked]:
        subj_id += 1
        # can be either "hbo", "hbr", or both
        for chroma in ["hbo", "hbr"]:
            data = deepcopy(subj_data.average(picks=chroma))
            value = data.crop(tmin=-1, tmax=15).data * 1.0e6
            
            # Reshape the data to be a flat numpy array
            value = np.reshape(value, -1)
            temporal_measurements.append(value)

            # Placeholder while we see if PCA gives better results
            avg_val = data.crop(tmin=-1, tmax=15).data.mean() * 1.0e6

            # Append metadata and extracted feature to dataframe
            this_df = pd.DataFrame(
                {'ID': subj_id, 'Chroma': chroma, 'Condition': evoked, 'Value': avg_val}, index=[0])
            df = pd.concat([df, this_df], ignore_index=True)


# df.reset_index(inplace=True, drop=True)
df['Value'] = pd.to_numeric(df['Value'])  # some Pandas have this as object
df

# You can export the dataframe for analysis in your favorite stats program
# df.to_csv("stats-export.csv")

,ID,Chroma,Condition,Value
0,1,hbo,Control,1.684122
1,1,hbr,Control,-0.570320
2,2,hbo,Control,0.254804
3,2,hbr,Control,-0.380185
4,3,hbo,Control,-0.013204
5,3,hbr,Control,2.376384
6,1,hbo,Inflammatory,1.050461
7,1,hbr,Inflammatory,-0.307104
8,2,hbo,Inflammatory,4.735021
9,2,hbr,Inflammatory,-1.830414


In [113]:
# temporal_measurements
# df
temporal_measurements = np.array(temporal_measurements)
temporal_measurements.shape

(24, 504)

In [121]:
# columns = []
# for i in range(len(temporal_measurements)):
#     print(i)
#     columns.append(f'Column-{i}')

# measurements_df = pd.DataFrame(temporal_measurements, columns=columns)
# measurements_df

# Use PCA in order to reduce the number of points in the temporal readings

## To do
- figure out if PCA is effective for the dataset or not
- Does the dataset need to be scaled (or should it normalized)
- Why are HBO and HBR the same value?

In [117]:
scaler = StandardScaler()
scaler.fit_transform(temporal_measurements)

temporal_measurements

pca = PCA(n_components=1)

reduced_measurements = pca.fit_transform(temporal_measurements)

df['Value'] = reduced_measurements

df


,ID,Chroma,Condition,Value
0,1,hbo,Control,21.004154
1,1,hbr,Control,21.004154
2,2,hbo,Control,-101.936464
3,2,hbr,Control,-101.936464
4,3,hbo,Control,19.412607
5,3,hbr,Control,19.412607
6,1,hbo,Inflammatory,18.501778
7,1,hbr,Inflammatory,18.501778
8,2,hbo,Inflammatory,2.624723
9,2,hbr,Inflammatory,2.624723


In [119]:
# Plot the Data
sns.catplot(x="Condition", y="Value", hue="ID", data=df.query("Chroma == 'hbo'"), ci=None, palette="muted", height=4, s=10)

In [120]:
# Inflammatory vs Neutral Prompt Analysis

input_data = df.query("Condition in ['Neutral', 'Inflammatory']")
input_data = input_data.query("Chroma in ['hbo']")

model = smf.mixedlm("Value ~ Condition", input_data, groups=input_data["ID"]).fit()
model.summary()

/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimizatio

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Mixed Linear Model Regression Results
===================================================================
Model:                 MixedLM    Dependent Variable:    Value     
No. Observations:      6          Method:                REML      
No. Groups:            3          Scale:                 16509.0306
Min. group size:       2          Log-Likelihood:        -26.2047  
Max. group size:       2          Converged:             No        
Mean group size:       2.0                                         
-------------------------------------------------------------------
                      Coef.  Std.Err.   z    P>|z|  [0.025   0.975]
-------------------------------------------------------------------
Intercept            -87.011   74.313 -1.171 0.242 -232.660  58.639
Condition[T.Neutral]  70.452  104.910  0.672 0.502 -135.167 276.071
Group Var             58.037  127.791                              
===================================================================

"""